# Review of robot kinematics and statics

## A robot jumper as a planar manipulator

![A jumping robot](jumper.png)

Let's define some coordinate systems

![](jumper_definitions.png)

First our imports

In [1]:
# symbolic computation tools
import sympy
from sympy import symbols, pprint
from sympy import sin, cos, asin, acos, pi
from sympy import Matrix

import numpy as np

from IPython import display # for the animation
import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()


# styling for plots
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
    


Define our symbolic variables that define our robot configuration

In [27]:
# We wrap in parentheses here so we can write it on multiple lines. Similar
# with the triple quotes on the string. Usually we don't need to use these things.
(theta0_sym, 
 theta1_sym, 
 theta2_sym, 
 link1_sym, 
 link2_sym, 
 link3_sym) = symbols(""" theta0_sym 
                         theta1_sym 
                         theta2_sym 
                         link1_sym 
                         link2_sym 
                         link3_sym """ , real = True)

Let's define our homogeneous transformation matrix that applies a rotation and a translation to vectors and matrices

In [28]:
def T(theta, x, y):
    """
    Function to return an arbitrary transformation matrix 
    This is for sympy symbolic calculation
    """
    return Matrix([[cos(theta), -sin(theta), x], 
                   [sin(theta), cos(theta), y],
                   [0, 0, 1]])

def sym_to_np(T):
    return np.array(T).astype(np.float64)

Define my transformation matrices for the links

In [29]:
T_01 = T(theta0_sym, 0, 0)
T_12 = T(theta1_sym, link1_sym, 0)
T_23 = T(theta2_sym, link2_sym, 0)

print(T_01)
print(T_12)
print(T_23)


Matrix([[cos(theta0_sym), -sin(theta0_sym), 0], [sin(theta0_sym), cos(theta0_sym), 0], [0, 0, 1]])
Matrix([[cos(theta1_sym), -sin(theta1_sym), link1_sym], [sin(theta1_sym), cos(theta1_sym), 0], [0, 0, 1]])
Matrix([[cos(theta2_sym), -sin(theta2_sym), link2_sym], [sin(theta2_sym), cos(theta2_sym), 0], [0, 0, 1]])


Define the point of the COM (x,y)

In [30]:
T_01*T_12*T_23

Matrix([
[(-sin(theta0_sym)*sin(theta1_sym) + cos(theta0_sym)*cos(theta1_sym))*cos(theta2_sym) + (-sin(theta0_sym)*cos(theta1_sym) - sin(theta1_sym)*cos(theta0_sym))*sin(theta2_sym), -(-sin(theta0_sym)*sin(theta1_sym) + cos(theta0_sym)*cos(theta1_sym))*sin(theta2_sym) + (-sin(theta0_sym)*cos(theta1_sym) - sin(theta1_sym)*cos(theta0_sym))*cos(theta2_sym), link1_sym*cos(theta0_sym) + link2_sym*(-sin(theta0_sym)*sin(theta1_sym) + cos(theta0_sym)*cos(theta1_sym))],
[ (-sin(theta0_sym)*sin(theta1_sym) + cos(theta0_sym)*cos(theta1_sym))*sin(theta2_sym) + (sin(theta0_sym)*cos(theta1_sym) + sin(theta1_sym)*cos(theta0_sym))*cos(theta2_sym),   (-sin(theta0_sym)*sin(theta1_sym) + cos(theta0_sym)*cos(theta1_sym))*cos(theta2_sym) - (sin(theta0_sym)*cos(theta1_sym) + sin(theta1_sym)*cos(theta0_sym))*sin(theta2_sym),  link1_sym*sin(theta0_sym) + link2_sym*(sin(theta0_sym)*cos(theta1_sym) + sin(theta1_sym)*cos(theta0_sym))],
[                                                                            

Whoof, that is not very "fun" looking. What if we try to simplify

In [31]:
sympy.simplify(T_01*T_12*T_23)

Matrix([
[cos(theta0_sym + theta1_sym + theta2_sym), -sin(theta0_sym + theta1_sym + theta2_sym), link1_sym*cos(theta0_sym) + link2_sym*cos(theta0_sym + theta1_sym)],
[sin(theta0_sym + theta1_sym + theta2_sym),  cos(theta0_sym + theta1_sym + theta2_sym), link1_sym*sin(theta0_sym) + link2_sym*sin(theta0_sym + theta1_sym)],
[                                        0,                                          0,                                                                  1]])

This makes much more sense, in fact it is exactly what we would have arrived at if we would have just written down the point (x,y) as a vector addition of the links. __But, what is critical to recognize is that by using the transformation matrices we can construct the point (x,y) in any frame we want by chaining together the appropriate matrices__

Let's draw our robot in different configurations

In [32]:
def draw_robot(ordered_list_of_transformations):
    """
    Draw the configuration of the three-link jumper.
    """


    # homogeneous unit vectors 
    x = np.array([[1, 0, 1]]).T 
    y = np.array([[0, 1, 1]]).T
    origin = np.array([[0, 0, 1]]).T

    prev_origin = np.array([[0, 0, 1]]).T    
    current_transformation = np.eye(3)

    plt.clf()
    ax = plt.gca()  # get current axis

    # now we plot
    plt.plot([prev_origin[0][0]], [prev_origin[1][0]], '-ko')
    plt.plot([prev_origin[0][0], x[0][0]], [prev_origin[1][0], x[1][0]], '-ro')
    plt.plot([prev_origin[0][0], y[0][0]], [prev_origin[1][0], y[1][0]], '-go')


    # loop the transforms in order of T_01, T_12, ... , T_N-1N
    for k, transform in enumerate(ordered_list_of_transformations):

        # update the transformation
        current_transformation = current_transformation @ transform
        new_origin = current_transformation @ origin
        new_x = current_transformation @ x
        new_y = current_transformation @ y

        # now we plot
        plt.plot([new_origin[0][0], new_x[0][0]], [new_origin[1][0], new_x[1][0]], '-r')
        plt.plot([new_origin[0][0], new_y[0][0]], [new_origin[1][0], new_y[1][0]], '-g')
        plt.plot([prev_origin[0][0], new_origin[0][0]], [prev_origin[1][0], new_origin[1][0]], '-ko')


#         ax.annotate('{'+str(k+1)+'}', xy=new_origin[0:2][0], xytext=new_origin[0:2][0])

        # lastly update the current_origin to the new_origin
        prev_origin = new_origin

    # lastly set the axes to be equal and appropriate
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')    

In [33]:
theta0 = 20 * np.pi/180 
theta1 = 140 * np.pi/180
theta2 = -90 * np.pi/180
link1 = 10
link2 = 20
link3 = 5

# T_01_real = T_np(theta0, 0, 0)
# T_12_real = T_np(theta1, link1, 0)
# T_23_real = T_np(theta2, link2, 0)
# T_34_real = T_np(0, link3, 0)

# # 'subs' substitutes in the numeric value for the variable, evalf evaluates the expression 
T_01_real = sym_to_np(T_01.subs([(theta0_sym, theta0)]))
T_12_real = sym_to_np(T_12.subs([(theta1_sym, theta1), (link1_sym, link1)]))
T_23_real = sym_to_np(T_23.subs([(theta2_sym, theta2), (link2_sym, link2)]))
T_34_real = sym_to_np(T(0, link3, 0))

plt.figure(figsize = (8,8))
draw_robot([T_01_real, T_12_real, T_23_real, T_34_real])

# sc = 25
# plt.axis([-sc,sc, -sc, sc])

plt.gcf()


<Figure size 1600x1572 with 1 Axes>

Let's make an animation

In [9]:
# final joint vals to arrive at
theta0 = 10 * np.pi/180 
theta1 = 70 * np.pi/180
theta2 = 90 * np.pi/180
link1 = 10
link2 = 20
link3 = 5

theta0_time = np.linspace(0, theta0, 50)
theta1_time = np.linspace(0, theta1, 50)
theta2_time = np.linspace(0, theta2, 50)

fig = plt.figure(figsize = (8,8))

for theta0_current, theta1_current, theta2_current in zip(theta0_time, theta1_time, theta2_time):
    # make t matrices
    T_01_real = sym_to_np(T_01.subs([(theta0_sym, theta0_current)]))
    T_12_real = sym_to_np(T_12.subs([(theta1_sym, theta1_current), (link1_sym, link1)]))
    T_23_real = sym_to_np(T_23.subs([(theta2_sym, theta2_current), (link2_sym, link2)]))
    T_34_real = sym_to_np(T(0, link3, 0))
    
    draw_robot([T_01_real, T_12_real, T_23_real, T_34_real])
    sc = 25
    plt.axis([-5,40, -5, 40])
    plt.draw()
    plt.pause(0.01)

    # Update the named display with a new figure
#     display.clear_output(wait=True)
#     display.display(plt.gcf())


__OK__, cool. But what was the point of this? Oh, yes we would like to be able to move our robot from a crouched configuration to a extended leg configuration as fast as possible. And more specifically, we would like to understand what the joint torques required to do this motion.

For this we need to dive into the jacobian


In [34]:
# x,y expressed in terms of the three joints and the three link lengths
T_01 = T(theta0_sym, 0, 0)
T_12 = T(theta1_sym, link1_sym, 0)
T_23 = T(theta2_sym, link2_sym, 0)

full_transformation = T_01*T_12*T_23*T(0,link3_sym,0)
full_transformation = sympy.simplify(full_transformation)
full_transformation

Matrix([
[cos(theta0_sym + theta1_sym + theta2_sym), -sin(theta0_sym + theta1_sym + theta2_sym), link1_sym*cos(theta0_sym) + link2_sym*cos(theta0_sym + theta1_sym) + link3_sym*cos(theta0_sym + theta1_sym + theta2_sym)],
[sin(theta0_sym + theta1_sym + theta2_sym),  cos(theta0_sym + theta1_sym + theta2_sym), link1_sym*sin(theta0_sym) + link2_sym*sin(theta0_sym + theta1_sym) + link3_sym*sin(theta0_sym + theta1_sym + theta2_sym)],
[                                        0,                                          0,                                                                                                                        1]])

The jacobian is defined as 

$$J = \begin{bmatrix}
\frac{\partial f_0}{\partial x_0} \dots \frac{\partial f_0}{\partial x_n} \\
\vdots \ddots \vdots \\
\frac{\partial f_m}{\partial x_0} \dots \frac{\partial f_m}{\partial x_n} \\
\end{bmatrix}
$$

and in the context of robot manipulators (and jumpers) it is used to relate the velocities of the joints with the velocity of a point on the robot (often the end-effector). $$ \dot{\mathbf{x}} = J \dot{\mathbf{q}} $$

where $ q = [\theta_0, \theta_1, \dots, \theta_n]^\intercal $ is the robot configuration. 

In [35]:

# first generate the equations for the position of the end mass. Can be by simply multiplying full_transform by [0, 0, 1].T
xyz_position = full_transformation @ np.array([[0,0,1]]).T
xyz_position = xyz_position[:2,:].simplify() # make simple

pprint(xyz_position, wrap_line = False)

J = xyz_position.jacobian([theta0_sym, theta1_sym, theta2_sym])

# for k in range(2):
#     J.append([])
#     J[k].append(xyz_position[k].diff(theta0_sym))
#     J[k].append(xyz_position[k].diff(theta1_sym))
#     J[k].append(xyz_position[k].diff(theta2_sym))

# J = Matrix(J)
J.simplify()

print("\r\n\r\n JACOBIAN MATRIX ")
pprint(J, wrap_line = False)

⎡link_1_sym⋅cos(θ_0_sym) + link_2_sym⋅cos(θ_0_sym + θ_1_sym) + link_3_sym⋅cos(θ_0_sym + θ_1_sym + θ_2_sym)⎤
⎢                                                                                                         ⎥
⎣link_1_sym⋅sin(θ_0_sym) + link_2_sym⋅sin(θ_0_sym + θ_1_sym) + link_3_sym⋅sin(θ_0_sym + θ_1_sym + θ_2_sym)⎦


 JACOBIAN MATRIX 
⎡-link_1_sym⋅sin(θ_0_sym) - link_2_sym⋅sin(θ_0_sym + θ_1_sym) - link_3_sym⋅sin(θ_0_sym + θ_1_sym + θ_2_sym)  -link_2_sym⋅sin(θ_0_sym + θ_1_sym) - link_3_sym⋅sin(θ_0_sym + θ_1_sym + θ_2_sym)  -link_3_sym⋅sin(θ_0_sym + θ_1_sym + θ_2_sym)⎤
⎢                                                                                                                                                                                                                                          ⎥
⎣link_1_sym⋅cos(θ_0_sym) + link_2_sym⋅cos(θ_0_sym + θ_1_sym) + link_3_sym⋅cos(θ_0_sym + θ_1_sym + θ_2_sym)   link_2_sym⋅cos(θ_0_sym + θ_1_sym) + link_3_sym⋅cos(θ_0_sym + θ_1_sym + θ_

In [36]:
print(J.shape)

(2, 3)


Note that the Jacobian is $m \times n$ in size where $n$ is the number of configuration variables and $m$ is the number of position variables we are tracking. The Jacobian transforms the rate of joint velocities into the rate of change of the output position and thus for any planay manipulator the Jacobian will alwyas be $2 \times n$ where $n$ is the number of joints in the manipulator. 

### Deriving the joints positions to command over time to jump along a defined straight line

First, imagine that link3 was of length 0 and so this was really only a two-link jumper with the mass concentrated at the end of link2. In this case the configuration variables would be $q = [\theta_0,  \theta_1]$ and the jacobian would be a square matrix representing the relationship 

$$ [\dot{x}, \dot{y}]^T = J [\dot{\theta_0}, \dot{\theta_1}]^T $$

Since the Jacobian was square we could attempt to invert it and solve for the unknown joint velocities if we had a desired velocity, $v_{des}$

$$ J^{-1} v_{des} = [\dot{\theta_0}, \dot{\theta_1}]^T$$


In [37]:
# make link3 = 0
full_transformation_two_link = T_01*T_12*T_23*T(0,0,0)
full_transformation_two_link = sympy.simplify(full_transformation_two_link)

# get FK
xyz_position_two_link = full_transformation_two_link*np.array([[0,0,1]]).T
xyz_position_two_link.simplify() # make simple
pprint(xyz_position_two_link)

⎡link_1_sym⋅cos(θ_0_sym) + link_2_sym⋅cos(θ_0_sym + θ_1_sym)⎤
⎢                                                           ⎥
⎢link_1_sym⋅sin(θ_0_sym) + link_2_sym⋅sin(θ_0_sym + θ_1_sym)⎥
⎢                                                           ⎥
⎣                             1                             ⎦


We see above that by making link3 length = 0,  $\theta_3$ naturally drops out of the forward kinematics

In [38]:
J = xyz_position_two_link[:2,:].jacobian([theta0_sym, theta1_sym, theta2_sym])
J

Matrix([
[-link1_sym*sin(theta0_sym) - link2_sym*sin(theta0_sym + theta1_sym), -link2_sym*sin(theta0_sym + theta1_sym), 0],
[ link1_sym*cos(theta0_sym) + link2_sym*cos(theta0_sym + theta1_sym),  link2_sym*cos(theta0_sym + theta1_sym), 0]])

Note now the last column is 0, which corresponds with our choice not to monitor $\theta_{3}$ so we can drop this

In [39]:
J = J[:,0:2]
pprint(J.shape)

J_inv = J.inv()
J_inv.simplify()
J_inv

(2, 2)


Matrix([
[(link2_sym*(-link1_sym*sin(theta0_sym) - link2_sym*sin(theta0_sym + theta1_sym))*cos(theta0_sym + theta1_sym) + link2_sym*(-link1_sym*cos(theta0_sym) - link2_sym*cos(theta0_sym + theta1_sym))*sin(theta0_sym + theta1_sym) + link2_sym*(link1_sym*cos(theta0_sym) + link2_sym*cos(theta0_sym + theta1_sym))*sin(theta0_sym + theta1_sym))/((-link1_sym*sin(theta0_sym) - link2_sym*sin(theta0_sym + theta1_sym))*(link2_sym*(-link1_sym*sin(theta0_sym) - link2_sym*sin(theta0_sym + theta1_sym))*cos(theta0_sym + theta1_sym) + link2_sym*(link1_sym*cos(theta0_sym) + link2_sym*cos(theta0_sym + theta1_sym))*sin(theta0_sym + theta1_sym))),                                link2_sym*sin(theta0_sym + theta1_sym)/(link2_sym*(-link1_sym*sin(theta0_sym) - link2_sym*sin(theta0_sym + theta1_sym))*cos(theta0_sym + theta1_sym) + link2_sym*(link1_sym*cos(theta0_sym) + link2_sym*cos(theta0_sym + theta1_sym))*sin(theta0_sym + theta1_sym))],
[                                                                      

Now we can attempt to use the Jacobian to specify the joint velocities for a given take off velocity

In [ ]:

J_links = J.subs([(link1_sym, link1), (link2_sym, link2), (link3_sym, link3)]).evalf()
J_inv = J_links.inv()

v_desired = np.array([[1, 2]]).T #45 degree jump

# start in crouched position 
theta0_current = 40*np.pi/180
theta1_current = 150*np.pi/180
theta2_current = 270*np.pi/180

# 'subs' substitutes in the numeric value for the variable, evalf evaluates the expression 
T_01_real = sym_to_np(T_01.subs([(theta0_sym, theta0_current)]))
T_12_real = sym_to_np(T_12.subs([(theta1_sym, theta1_current), (link1_sym, link1)]))
T_23_real = sym_to_np(T_23.subs([(theta2_sym, theta2_current), (link2_sym, link2)]))

plt.figure(0,figsize = (8,8))
draw_robot([T_01_real, T_12_real, T_23_real])

start_xy = sym_to_np(T_01_real @ T_12_real @ T_23_real @ np.array([[0,0,1]]).T)

# step size
eps = 0.05
for k in range(500):
        
    # for drawing
    T_01_real = sym_to_np(T_01.subs([(theta0_sym, theta0_current)]))
    T_12_real = sym_to_np(T_12.subs([(theta1_sym, theta1_current), (link1_sym, link1)]))
    T_23_real = sym_to_np(T_23.subs([(theta2_sym, theta2_current), (link2_sym, link2)]))


    draw_robot([T_01_real, T_12_real, T_23_real])
    sc = 25
    plt.axis([-sc,sc, -1, sc])
    plt.plot([start_xy[0][0], start_xy[0][0]+v_desired[0][0]*100], 
             [start_xy[1][0], start_xy[1][0]+v_desired[1][0]*100], '--')

    plt.draw()
    plt.pause(0.01)
    
    # update
    J_inv_current = J_inv.subs([(theta0_sym, theta0_current),(theta1_sym, theta1_current)])
    
    # compute next motion
    delta_q = J_inv_current * v_desired
    
    theta0_current = theta0_current + delta_q[0]*eps
    theta1_current = theta1_current + delta_q[1]*eps
    


## Full pose tracking of the COM position and orientation using the jacobian

Now imagine $l_3$ is not 0, then we can track both the position and orientation of the COM and body. The forward kinematics now need to generate 

$$f(\theta_1, \theta_2, \theta_3) = [x_{COM}, y_{COM}, \theta_{COM}]^T$$

and thus the jacobian equation will be 

$$[\dot{x}_{COM}, \dot{y}_{COM}, \dot{\theta}_{COM}]^T = \mathbf{J} ~ [\dot{\theta}_0, \dot{\theta}_1, \dot{\theta}_2]^T $$

Note $\theta_{COM} = \theta_0 + \theta_1 + \theta_2$

In [52]:
full_transformation = sympy.simplify(full_transformation)
xyz_position_two_link = full_transformation @ np.array([[0,0,1]]).T
# print(xyz_position_two_link)

forward_kin = Matrix([xyz_position_two_link[0,0], xyz_position_two_link[1,0], theta0_sym + theta1_sym + theta2_sym])
forward_kin

Matrix([
[link1_sym*cos(theta0_sym) + link2_sym*cos(theta0_sym + theta1_sym) + link3_sym*cos(theta0_sym + theta1_sym + theta2_sym)],
[link1_sym*sin(theta0_sym) + link2_sym*sin(theta0_sym + theta1_sym) + link3_sym*sin(theta0_sym + theta1_sym + theta2_sym)],
[                                                                                    theta0_sym + theta1_sym + theta2_sym]])

In [56]:
J = forward_kin.jacobian([theta0_sym, theta1_sym, theta2_sym])
J

Matrix([
[-link1_sym*sin(theta0_sym) - link2_sym*sin(theta0_sym + theta1_sym) - link3_sym*sin(theta0_sym + theta1_sym + theta2_sym), -link2_sym*sin(theta0_sym + theta1_sym) - link3_sym*sin(theta0_sym + theta1_sym + theta2_sym), -link3_sym*sin(theta0_sym + theta1_sym + theta2_sym)],
[ link1_sym*cos(theta0_sym) + link2_sym*cos(theta0_sym + theta1_sym) + link3_sym*cos(theta0_sym + theta1_sym + theta2_sym),  link2_sym*cos(theta0_sym + theta1_sym) + link3_sym*cos(theta0_sym + theta1_sym + theta2_sym),  link3_sym*cos(theta0_sym + theta1_sym + theta2_sym)],
[                                                                                                                        1,                                                                                             1,                                                    1]])

In [63]:

J_links = J.subs([(link1_sym, link1), (link2_sym, link2), (link3_sym, link3)]).evalf()
J_inv = J_links.inv()

v_desired = np.array([[1, 2, 0]]).T #Don't rotate body COM

# start in crouched position 
theta0_current = 40*np.pi/180
theta1_current = 150*np.pi/180
theta2_current = 270*np.pi/180

# 'subs' substitutes in the numeric value for the variable, evalf evaluates the expression 
T_01_real = sym_to_np(T_01.subs([(theta0_sym, theta0_current)]))
T_12_real = sym_to_np(T_12.subs([(theta1_sym, theta1_current), (link1_sym, link1)]))
T_23_real = sym_to_np(T_23.subs([(theta2_sym, theta2_current), (link2_sym, link2)]))

plt.figure(0,figsize = (8,8))
draw_robot([T_01_real, T_12_real, T_23_real])

start_xy = sym_to_np(T_01_real @ T_12_real @ T_23_real @ np.array([[0,0,1]]).T)

# step size
eps = 0.05
for k in range(500):
        
    # for drawing
    T_01_real = sym_to_np(T_01.subs([(theta0_sym, theta0_current)]))
    T_12_real = sym_to_np(T_12.subs([(theta1_sym, theta1_current), (link1_sym, link1)]))
    T_23_real = sym_to_np(T_23.subs([(theta2_sym, theta2_current), (link2_sym, link2)]))
    T_34_real = sym_to_np(T(0, link3, 0))

    draw_robot([T_01_real, T_12_real, T_23_real, T_34_real])
    sc = 25
    plt.axis([-sc,sc, -1, sc])
    plt.plot([start_xy[0][0], start_xy[0][0]+v_desired[0][0]*100], 
             [start_xy[1][0], start_xy[1][0]+v_desired[1][0]*100], '--')

    plt.draw()
    plt.pause(0.01)
    
    # update
    J_inv_current = J_inv.subs([(theta0_sym, theta0_current),(theta1_sym, theta1_current), (theta2_sym, theta2_current)])
    
    # compute next motion
    delta_q = J_inv_current * v_desired
    
    theta0_current = theta0_current + delta_q[0]*eps
    theta1_current = theta1_current + delta_q[1]*eps
    theta2_current = theta2_current + delta_q[2]*eps
